In [ ]:
import re
import nltk
import numpy as np
import random
import string


nltk.download("punkt")
with open("movie_lines.txt/movie_lines.txt", "r", encoding="iso-8859-1") as archivo:
    data = archivo.read()

print(data[:5])

In [ ]:
prueba = "L2457 +++$+++ u16 +++$+++ m1 +++$+++ COLUMBUS +++$+++ Thank you."
pattern = re.match(r'L(.*\+\+\+\$\+\+\+){4} ', prueba)
print(pattern)
prueba_sin = re.split(r'L(.*\+\+\+\$\+\+\+){4} ', prueba)
print(prueba_sin)
prueba_sin2 = re.sub(r'L(.*\+\+\+\$\+\+\+){4} ',"",prueba)
print(prueba_sin2)

In [ ]:
data1 = re.sub(r'L(.*\+\+\+\$\+\+\+){4} ',"",data)
print(data1[:1000])
data1 = data1[:10000]

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [ ]:
tokens = nltk.word_tokenize(data1.lower())
input_sequence = []
output_sequence = []
words = []
ignore_words = ["?", "!", ".", ",", "--"]
for i in range(len(tokens)-1):
    input_sequence.append(tokens[i])
    output_sequence.append(tokens[i+1])
    words.append(tokens[i])
words.append(tokens[len(tokens)-1])
    
print(input_sequence[:30])
print(words[:30])
input_sequence1 = [lemmatizer.lemmatize(word.lower()) for word in input_sequence if word not in ignore_words]
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
output_sequence1 = [lemmatizer.lemmatize(word.lower()) for word in output_sequence if word not in ignore_words]
print(input_sequence1[:30])
print(words[:30])
word_2index = {}
for i,w in enumerate(words):
    word_2index[w] = i
    
onehot_input_sequence = []
for word in input_sequence1:
    vector = np.zeros(len(words))
    vector[word_2index[word.lower()]] = 1
    onehot_input_sequence.append(vector)
    
onehot_output_sequence = []
for word in output_sequence1:
    vector = np.zeros(len(words))
    vector[word_2index[word.lower()]] = 1
    onehot_output_sequence.append(vector)
    
print(onehot_input_sequence[:30])       
input_sequence_array = np.array(onehot_input_sequence)
output_sequence_array = np.array(onehot_output_sequence)
print(input_sequence_array[:30])
print(len(input_sequence_array))
print(len(output_sequence_array))
print(len(tokens))
print(tokens[:50])
print(words[:50])

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding


model = Sequential()
model.add(Embedding(input_dim= len(words), output_dim=100))
model.add(LSTM(256))
model.add(Dense(len(words), activation= "softmax"))
model.compile(loss = "categorical_crossentropy", optimizer= "adam", metrics = ["accuracy"])



In [ ]:
model.fit(input_sequence_array, output_sequence_array, epochs=100)

In [ ]:
def generarRespuesta(entrada):
    sec_entrada = nltk.word_tokenize(entrada.lower())
    words_entrada = [lemmatizer.lemmatize(w.lower()) for w in sec_entrada]
    onehot_entrada = []
    for word in words_entrada:
        vector_entrada = np.zeros(len(words))
        vector_entrada[word_2index[word.lower()]] = 1
        onehot_entrada.append(vector_entrada)  
    pred_one_hot = model.pred(onehot_entrada)
    index_2words = {}
    for i,w in enumerate(words):
        index_2words[i] = w
    pos = pred_one_hot.index(1)
    pred = index_2words[pos]  
    return pred

In [ ]:
entrada = input("Yo: ")
len_entrada = len(entrada)
for i in range (0,20):
    pred = generarRespuesta(entrada)
    entrada += pred
print("Bot: " + entrada[len_entrada:])